In [2]:
import pandas as pd
import numpy as np
import numpy.core.defchararray as npd
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',1000000)

organ_offers = pd.read_csv("/Users/james/Documents/OneLegacy/organoffers.csv", low_memory = False)

organ_offers.loc[:, 'accepted_organs'] = ''

In [3]:
donors = organ_offers.groupby('donor_id')
trial = donors.get_group('AGIC169')


In [5]:
def get_organs(trial):
    reneg_inds1 = trial['initial_response']=='Z' 
    reneg_inds2 = trial['donor_cross_clamp_datetime'] < trial['datetime_refusal_entered']
    reneg_inds3 = trial['initial_response_datetime'] < trial['donor_cross_clamp_datetime']
    reneg_inds4 = trial['offer_accepted']=='N'
    reneg_inds = reneg_inds1 & reneg_inds2 & reneg_inds3 & reneg_inds4
    accept_inds = trial['offer_accepted']=='Y'
    trial.loc[:, 'reneg'] = 0
    trial.loc[reneg_inds | accept_inds, 'reneg'] = 1
    trial.loc[:, 'accepted_organs'] = ''

    trial.head()
    accepted_organs = []
    current = set()
    for i in range(len(trial)):
        if (trial.reneg.iloc[i] == 1):
            current.add(trial.match_organ.iloc[i])
            dummy = current.copy()
            accepted_organs.append(dummy)
        else: 
            dummy = current.copy()
            accepted_organs.append(dummy)
    return accepted_organs


In [9]:
time_delta = 15*60
counter = 0
auc_data = pd.DataFrame(columns = ['allocated_organs', 'time_in_seconds', 'donor_age', 'donor_abo', 'donor_gender', 'donor_kdpi', 'donor_pf_ratio', 'available_organs'])
for name, group in donors:
    print(name, counter)
    donor_age = group.donor_age_in_months.iloc[0]
    donor_abo = group.donor_abo.iloc[0]
    donor_gender = group.donor_gender.iloc[0]
    donor_kdpi = group.donor_kdpi.iloc[0]
    donor_pf_ratio = group.donor_pf_ratio.iloc[0]
    donor_organs = group.match_organ.unique()
    group = group.sort_values('initial_response_datetime')
    init = group.initial_response_datetime.iloc[0]
    times = (pd.to_datetime(group.initial_response_datetime) - pd.to_datetime(init)).dt.total_seconds()
    a_o = get_organs(group)
    cur_list = []
    step0 = [a_o[0], 0, donor_age, donor_abo, donor_gender, donor_kdpi, donor_pf_ratio, donor_organs]
    cur_list.append(step0)
    auc_data = pd.concat([auc_data, pd.Series(step0)])
    for i in range(len(group)):
        if (times.iloc[i] > time_delta*len(cur_list)):
            step = [a_o[i-1], time_delta*len(cur_list), donor_age, donor_abo, donor_gender, donor_kdpi, donor_pf_ratio, donor_organs]
            cur_list.append(step)
            auc_data = pd.concat([auc_data, pd.Series(step)])
    counter = counter + 1
    

AGC1095 0
AGC1152 1
AGC1231 2
AGC1347 3
AGC2028 4
AGC2262 5
AGC3065 6
AGC3119 7
AGC3340 8
AGC3398 9
AGC3471 10
AGC4178 11
AGC5169 12
AGCK031 13
AGCK154 14
AGCK273 15
AGCK277 16
AGCL192 17
AGCL219 18
AGCL377 19
AGCM052 20
AGCM059 21
AGCM164 22
AGCM258 23
AGCM345 24
AGCN156 25
AGCN197 26
AGCN202 27
AGCN205 28
AGCN437 29
AGCN467 30
AGCO205 31
AGCO294 32
AGCO469 33
AGCP344 34
AGCP437 35
AGCP476 36
AGCQ165 37
AGCQ169 38
AGCQ335 39
AGCR049 40
AGCS032 41
AGCS094 42
AGCS170 43
AGCS274 44
AGCS360 45
AGCS372 46
AGCS467 47
AGCS489 48
AGCU015 49
AGCU032 50
AGCU343 51
AGCU346 52
AGCV197 53
AGCV392 54
AGCV472 55
AGCW012 56
AGCW160 57
AGCW192 58
AGCW459 59
AGCX062 60
AGCY030 61
AGCY151 62
AGCY166 63
AGCY174 64
AGCY255 65
AGCY258 66
AGCY428 67
AGD1198 68
AGD1318 69
AGD2137 70
AGD2320 71
AGD3008 72
AGD4004 73
AGD4136 74
AGD4360 75
AGD4390 76
AGDB032 77
AGDB124 78
AGDB218 79
AGDB380 80
AGDB383 81
AGDB423 82
AGDB497 83
AGDC086 84
AGDC149 85
AGDC206 86
AGDC327 87
AGDD127 88
AGDD128 89
AGDD260 90
AGDE394 9

KeyboardInterrupt: 

In [14]:
time_delta = 15*60
counter = 0
auc_data = []
for name, group in donors:
    if (counter%100 == 1): print(counter)
    donor_id = group.donor_id.iloc[0]
    donor_age = group.donor_age_in_months.iloc[0]
    donor_abo = group.donor_abo.iloc[0]
    donor_gender = group.donor_gender.iloc[0]
    donor_kdpi = group.donor_kdpi.iloc[0]
    donor_pf_ratio = group.donor_pf_ratio.iloc[0]
    donor_organs = group.match_organ.unique()
    group = group.sort_values('initial_response_datetime')
    init = group.initial_response_datetime.iloc[0]
    times = (pd.to_datetime(group.initial_response_datetime) - pd.to_datetime(init)).dt.total_seconds()
    a_o = get_organs(group)
    cur_list = []
    dict1 = {'donor_id':donor_id, 'allocated_organs': a_o[0], 'time_in_seconds':0, 'donor_age':donor_age, 'donor_abo':donor_abo, 'donor_gender':donor_gender, 'donor_kdpi':donor_kdpi, 'donor_pf_ratio':donor_pf_ratio, 'available_organs':donor_organs}
    cur_list.append(dict1)
    auc_data.append(dict1)
    for i in range(len(group)):
        if (times.iloc[i] > time_delta*len(cur_list)):
            step = {'donor_id': donor_id, 'allocated_organs': a_o[0], 'time_in_seconds':time_delta*len(cur_list)/60, 'donor_age':donor_age, 'donor_abo':donor_abo, 'donor_gender':donor_gender, 'donor_kdpi':donor_kdpi, 'donor_pf_ratio':donor_pf_ratio, 'available_organs':donor_organs}
            cur_list.append(step)
            auc_data.append(step)
    counter = counter + 1
    

1
101
201
301
401
501


In [15]:
auc_data_clean = pd.DataFrame(auc_data)
auc_data_clean.head()

,donor_id,allocated_organs,time_in_seconds,donor_age,donor_abo,donor_gender,donor_kdpi,donor_pf_ratio,available_organs
0,AGC1095,{},0.0,534,O,F,25,560.0,"[HL, IN, KI, KP, LI, LU]"
1,AGC1095,{},15.0,534,O,F,25,560.0,"[HL, IN, KI, KP, LI, LU]"
2,AGC1095,{},30.0,534,O,F,25,560.0,"[HL, IN, KI, KP, LI, LU]"
3,AGC1095,{},45.0,534,O,F,25,560.0,"[HL, IN, KI, KP, LI, LU]"
4,AGC1095,{},60.0,534,O,F,25,560.0,"[HL, IN, KI, KP, LI, LU]"
